In [1]:
import os
os.chdir('../')

### from "sample noise data pair by Euler and save pt.ipynb"

In [2]:
import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_NFE=120_N=256.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64])


In [3]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False

###############################################################################
# 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
###############################################################################
sys.argv = [
    "main.py",
    "--config", "imagenet64.yml",  # 사용하려는 config
    "--sample",
    "--sample_type", "rbfsolverglq",
    "--dpm_solver_type", "data_prediction",
    "--dpm_solver_order", "3",
    "--timesteps", "5",
    "--skip_type", "logSNR",
    "--log_scale_min", "-6.0",
    "--log_scale_max", "6.0",
    "--log_scale_num", "100",
    "--scale_dir", "/data/data/rbfsolverglq",
    "--ni"
]

###############################################################################
# 2) 인자/설정 로드
###############################################################################
args, config = parse_args_and_config()

###############################################################################
# 3) Diffusion 객체 생성 -> 모델 로딩
###############################################################################
diffusion = Diffusion(args, config, rank=0)
diffusion.prepare_model()
diffusion.model.eval()

###############################################################################
# 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
###############################################################################
device = diffusion.device


INFO - main.py - 2025-03-28 06:07:17,252 - Using device: cuda


[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


In [3]:
!mkdir /data/data/rbfsolverglq

In [7]:
n = 256
i = 0
while True:    
    start = i*n
    end = (i+1)*n
    if end > len(noise):
        break
    noise_batch = noise[start:end].to(device)
    target_batch = target[start:end].to(device)
    print(noise_batch.shape)
    (pred, optimal_gammas), _ = diffusion.sample_image(noise_batch, diffusion.model, target=target_batch, exp_num=i)
    loss = F.mse_loss(pred, target_batch)
    print('loss :', loss)
    print(optimal_gammas)
    i += 1


torch.Size([256, 3, 64, 64])
/data/data/rbfsolverglq/NFE=5,p=3,exp_num=0.npy  saved!
loss : tensor(0.0361, device='cuda:0')
[[-6.          0.06060606  0.54545455  0.18181818 -6.        ]
 [ 5.51515152 -0.18181818 -0.06060606 -0.06060606  0.        ]]


### Batch

In [4]:
import sys
import torch
import numpy as np
from main import parse_args_and_config, Diffusion
from tqdm import tqdm

import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_NFE=120_N=256.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

model = None
for NFE in tqdm([5, 10, 15, 25]):
    sys.argv = [
        "main.py",
        "--config", "imagenet64.yml",  # 사용하려는 config
        "--sample",
        "--sample_type", "rbfsolverglq",
        "--dpm_solver_type", "data_prediction",
        "--dpm_solver_order", "3",
        "--timesteps", str(NFE),
        "--skip_type", "logSNR",
        "--log_scale_min", "-6.0",
        "--log_scale_max", "6.0",
        "--log_scale_num", "100",
        "--scale_dir", "/data/data/rbfsolverglq",
        "--exp_num", "0",
        "--verbose", "critical",
        "--ni",
    ]

    args, config = parse_args_and_config()
    diffusion = Diffusion(args, config, rank=0)

    if model is None:
        diffusion.prepare_model()
        diffusion.model.eval()
        model = diffusion.model
    device = diffusion.device

    n = 256
    noise_batch = noise[:n].to(device)
    target_batch = target[:n].to(device)
    (pred, _), _ = diffusion.sample_image(noise_batch, model, target=target_batch)
    print(F.mse_loss(pred, target_batch))


torch.Size([256, 3, 64, 64]) torch.Size([256, 3, 64, 64])


  0%|          | 0/4 [00:00<?, ?it/s]

[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


 25%|██▌       | 1/4 [00:03<00:11,  3.79s/it]

/data/data/rbfsolverglq/NFE=5,p=3,exp_num=0.npy  saved!
tensor(0.0329, device='cuda:0')


...

/data/data/rbfsolverglq/NFE=10,p=3,exp_num=0.npy  saved!
tensor(0.0056, device='cuda:0')


 75%|███████▌  | 3/4 [00:18<00:06,  6.75s/it]

/data/data/rbfsolverglq/NFE=15,p=3,exp_num=0.npy  saved!
tensor(0.0034, device='cuda:0')


100%|██████████| 4/4 [00:33<00:00,  8.30s/it]

/data/data/rbfsolverglq/NFE=25,p=3,exp_num=0.npy  saved!
tensor(0.0044, device='cuda:0')


In [9]:
print('done')

done
